## Library import

In [1]:
import os
import itertools
from random import randint
import json
import glob
import numpy as np
import pandas as pd
import librosa
import torch 
import torch.utils.data as data
import torchvision.transforms as transforms
import lightning as L

## Establishing folder structure for dataset 
data_split -> sub -> audio_material -> type_of_file.

In [2]:
file_ext = "*.npy"

In [3]:
file_list = glob.glob(os.path.join(dir_pth, file_ext))

NameError: name 'dir_pth' is not defined

In [4]:
def data_segmentation(fpath : str):
  fname = (fpath.split("/")[-1])
  file_seg = (fpath.split("/")[-1]).split("_-_")[0]
  print(fname)

  # creating sub-dirs : train, val and test
  for seg in ["train", "val", "test"]:
    sub_dir = os.path.join(dir_pth, seg)
    if not os.path.exists(sub_dir):
      os.makedirs(sub_dir)

  # moving file to desired segment folder
  src_file = fpath
  dest_file = os.path.join(dir_pth, file_seg, fname)
  shutil.move(src_file, dest_file)

  # Check if the file has been moved
  if os.path.exists(dest_file):
    print(f"File has been moved to : {dest_file}")
  else:
    print("File move operation failed")

In [5]:
for f in file_list:
  data_segmentation(f)

NameError: name 'file_list' is not defined

## Data Loader

In [5]:
data_dir = '/Users/klvijeth/Documents/ML-project/toy_split'

In [6]:
test_eeg_file_path = '/Users/klvijeth/Documents/ML-project/toy_split/test/test_-_sub-001_-_audiobook_1_-_eeg.npy'
test_mel_file_path = '/Users/klvijeth/Documents/ML-project/toy_split/test/test_-_sub-001_-_audiobook_1_-_mel.npy'

In [7]:
def sliding_window(self, x: torch.Tensor) -> torch.Tensor:
        return x.unfold(0, self.frame_length, self.hop_length)

In [8]:
## test sliding window
try_eeg = torch.from_numpy(np.load(test_eeg_file_path))
try_mel= torch.from_numpy(np.load(test_mel_file_path))
print("EEG: ",) 

EEG: 


### Custom PyTorch Dataset

In [9]:
class EEGDataset(data.Dataset):
    def __init__(self, files, input_length, channels, task, g_con = True) -> None:
        super().__init__()
        self.input_length = input_length
        self.files = self.group_recordings(files)
        self.channels = channels
        self.task = task
        self.g_con = g_con

    def group_recordings(self, files):
        new_files = []
        grouped = itertools.groupby(sorted(files), lambda x: "_-_".join(os.path.basename(x).split("_-_")[:3]))
        for recording_name, feature_paths in grouped:
            new_files += [sorted(feature_paths, key=lambda x: "0" if x == "eeg" else x)]
        return new_files
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, recording_index):
        if self.task !="test":
            framed_data = []
            for idx, feature in enumerate(self.files[recording_index]):
                data = np.load(feature)
                if idx == 0:
                    start_idx = randint(0, len(data) - self.input_length)
                    
                framed_data += [data[start_idx:start_idx + self.input_length]]
            if self.g_con == True:
                sub_id = feature.split('/')[-1].split('_-_')[1].split('-')[-1]
                sub_id = int(sub_id) - 1
            else:
                sub_id = torch.FloatTensor([0])
            return torch.FloatTensor(framed_data[0]), torch.FloatTensor(framed_data[1])
        else:
            for idx, feature in enumerate(self.files[recording_index]):
                data = np.load(feature)
                nsegment = data.shape[0] // self.input_length
                data = data[:int(nsegment * self.input_length)]
                segment_data = [torch.FloatTensor(data[i:i+self.input_length]).unsqueeze(0) for i in range(0, data.shape[0], self.input_length)]
                segment_data = torch.cat(segment_data)
                framed_data += [segment_data]
            
            if self.g_con == True:
                sub_idx = feature.split('/')[-1].split('_-_')[1].split('-')[-1]
                sub_idx = int(sub_idx) - 1    
    
            else:
                sub_idx = torch.FloatTensor([0])
            return framed_data[0], framed_data[1]

### Lightning DataModule

In [11]:
def pearson_correlation(y_true, y_pred, axis=1):

    y_true_mean = torch.mean(y_true, dim=axis, keepdim=True)
    y_pred_mean = torch.mean(y_pred, dim=axis, keepdim=True)

    # Compute the numerator and denominator of the pearson correlation.
    numerator = torch.sum((y_true - y_true_mean) * (y_pred - y_pred_mean),
        dim=axis,
        keepdim=False)

    std_true = torch.sum((y_true - y_true_mean)**2, dim=axis, keepdim=False)
    std_pred = torch.sum((y_pred - y_pred_mean)**2, dim=axis, keepdim=False)
    denominator = torch.sqrt(std_true * std_pred)
    
    pearsonR = torch.div(numerator, denominator + 1e-6)

    assert torch.all(torch.lt(pearsonR, 1)) and torch.all(torch.gt(pearsonR, -1)), "Loss contains values outside the range of -1 to 1"

    return pearsonR


def pearson_loss(y_true, y_pred, axis=1):
    return -pearson_correlation(y_true, y_pred, axis=axis)

def pearson_metric(y_true, y_pred, axis=1):
    return pearson_correlation(y_true, y_pred, axis=axis)
    
def l1_loss(y_true, y_pred, axis=1):
    l1_dist = torch.abs(y_true - y_pred)
    l1_loss = torch.mean(l1_dist, axis = axis, keepdim=False)
    return l1_loss

In [59]:
import torch
import itertools
import os
import numpy as np
from torch.utils.data import Dataset
import pdb
from random import randint

class RegressionDataset(Dataset):
    """Generate data for the regression task."""

    def __init__(
        self,
        files,
        input_length,
        channels,
        task,
        g_con = True
    ):

        self.input_length = input_length
        self.files = self.group_recordings(files)
        self.channels = channels
        self.task = task
        self.g_con = g_con

    def group_recordings(self, files):
 
        new_files = []
        grouped = itertools.groupby(sorted(files), lambda x: "_-_".join(os.path.basename(x).split("_-_")[:3]))
        for recording_name, feature_paths in grouped:
            new_files += [sorted(feature_paths, key=lambda x: "0" if x == "eeg" else x)]

        return new_files

    def __len__(self):
        return len(self.files)

    def __getitem__(self, recording_index):
        
        # 1. For within subject, return eeg, envelope and subject ID
        # 2. For held-out subject, return eeg, envelope

        if self.task == "train":
            x, y = self.__train_data__(recording_index)

        else:
            x, y = self.__test_data__(recording_index)

        return x, y


    def __train_data__(self, recording_index):

        framed_data = []

        for idx, feature in enumerate(self.files[recording_index]):
            data = np.load(feature)

            if idx == 0: 
                start_idx= randint(0,len(data)- self.input_length)

            framed_data += [data[start_idx:start_idx + self.input_length]]

        if self.g_con == True:
            sub_idx = feature.split('/')[-1].split('_-_')[1].split('-')[-1]
            sub_idx = int(sub_idx) - 1 
    
        else:
            sub_idx = torch.FloatTensor([0])
    
            # return torch.FloatTensor(framed_data[0]), torch.FloatTensor(framed_data[1]), sub_idx
        
            
        return torch.FloatTensor(framed_data[0]), torch.FloatTensor(framed_data[1])

    def __test_data__(self, recording_index):
        """
        return: list of segments [[eeg, envelope] ...] depending on self.input_length 
                e.g.,for 10 second-long input signal and input_length==5, return [[5, 5], [5, 5]]
        
        """
        framed_data = []

        for idx, feature in enumerate(self.files[recording_index]):
            data = np.load(feature)
            nsegment = data.shape[0] // self.input_length
            data = data[:int(nsegment * self.input_length)]
            segment_data = [torch.FloatTensor(data[i:i+self.input_length]).unsqueeze(0) for i in range(0, data.shape[0], self.input_length)]
            segment_data = torch.cat(segment_data)
            framed_data += [segment_data]
            
        if self.g_con == True:
            sub_idx = feature.split('/')[-1].split('_-_')[1].split('-')[-1]
            sub_idx = int(sub_idx) - 1    

        else:
            sub_idx = torch.FloatTensor([0])

        return framed_data[0], framed_data[1]

In [60]:
features = ["eeg"] + ["mel"]
path="/Users/klvijeth/Documents/ML-project/toy_split/train/"

In [99]:
import torch
import torch.nn as nn

class YourModel(nn.Module):
    def __init__(self):
        super(YourModel, self).__init__()
        self.fc = nn.Linear(128, 10)
        self.conv = nn.Conv1d(64, 128, kernel_size = 7, padding=3)

    def forward(self, dec_input):
        dec_output = self.conv(dec_input.transpose(1,2))
        dec_output = dec_output.transpose(1,2)
        #print(dec_output.size())
        dec_output=self.fc(dec_output)
        return dec_output

# Assuming nb_channels, nb_filters, and integration_window are defined
nb_channels = 64
nb_filters = 1
integration_window = 32

# Create an instance of the model
model = YourModel()

# Print the model architecture
print(model)

YourModel(
  (fc): Linear(in_features=128, out_features=10, bias=True)
  (conv): Conv1d(64, 128, kernel_size=(7,), stride=(1,), padding=(3,))
)


In [108]:
folder_path="/Users/klvijeth/Documents/ML-project/toy_split"

In [109]:
optimizer = torch.optim.Adam(model.parameters(),
                                lr=0.0005,
                                betas=(0.9, 0.98),
                                eps=1e-09)

In [110]:
train_files = [x for x in glob.glob(os.path.join(folder_path+"/train/","train_-_*")) if os.path.basename(x).split("_-_")[-1].split(".")[0] in features]
train_set= RegressionDataset(train_files, 640, 64, 'train', False)
print(train_set)

In [111]:
train_dataloader = torch.utils.data.DataLoader(
            train_set,
            batch_size = 1,
            num_workers = 0,
            sampler = None,
            drop_last=True,
            shuffle=True)
print(len(train_dataloader))

2


In [112]:
val_files = [x for x in glob.glob(os.path.join(folder_path+"/val/", "val_-_*")) if os.path.basename(x).split("_-_")[-1].split(".")[0] in features]
val_set = RegressionDataset(val_files, 640, 64, 'val', False)

In [113]:
val_dataloader = torch.utils.data.DataLoader(
        val_set,
        batch_size = 1,
        num_workers = 0,
        sampler = None,
        drop_last=True,
        shuffle=False)
print(len(val_dataloader))

3


In [114]:
test_files = [x for x in glob.glob(os.path.join(folder_path+"/test/", "test_-_*")) if os.path.basename(x).split("_-_")[-1].split(".")[0] in features]
test_set = RegressionDataset(test_files, 640, 64, 'test', False)

In [115]:
test_dataloader = torch.utils.data.DataLoader(
        test_set,
        batch_size = 1,
        num_workers = 0,
        sampler = None,
        drop_last=True,
        shuffle=False)

In [116]:
from torch.optim.lr_scheduler import StepLR
scheduler = StepLR(optimizer, step_size=50, gamma=0.9)
#writer = get_writer(result_folder, experiment_folder)

In [117]:
torch.device(f"cuda:{args.gpu}" if torch.cuda.is_available() else "cpu")

device(type='cpu')

In [118]:
print(model)
for epoch in range(10):
    print("Epoch",epoch+1)
    model.train()
    train_loss = 0

    for inputs, labels in train_dataloader:
        optimizer.zero_grad()

        inputs = inputs.to(device)
        labels = labels.to(device)
        #sub_id = sub_id.to(device)
        outputs = model(inputs)

        l_p = pearson_loss(outputs, labels) 
        l_1 = l1_loss(outputs, labels)
        loss = l_p + 0.2 * l_1
        loss = loss.mean()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_dataloader)

    if epoch % 1 == 0:
        print(f'|-Train-|{epoch}: {train_loss:.3f}')
        #writer.add_losses("Loss", "train",  train_loss, epoch)
        #writer.add_losses("Loss_l1", "train",  train_loss, epoch)

    # Validate the model.
    val_loss = 0
    val_metric = 0

    if 1:

        model.eval()

        with torch.no_grad():
            for val_inputs, val_labels in val_dataloader:
                val_inputs = val_inputs.squeeze(0).to(device)
                val_labels = val_labels.squeeze(0).to(device)
                #val_sub_id = val_sub_id.to(device)

                val_outputs = model(val_inputs)
                val_loss   += pearson_loss(val_outputs, val_labels).mean()
                val_metric += pearson_metric(val_outputs, val_labels).mean()

            val_loss /= len(val_dataloader)
            val_metric /= len(val_dataloader)
            val_metric = val_metric.mean()

            print(f'|-Validation-|{epoch}: {val_loss.mean().item():.3f} {val_metric.item():.3f}')
            #writer.add_losses("Loss", "Validation",  val_loss, epoch)
            #writer.add_losses("Pearson", "Validation",  val_metric, epoch)

            # Test the model.
            test_loss = 0
            test_metric = 0

            for test_inputs, test_labels in test_dataloader:
                test_inputs = test_inputs.squeeze(0).to(device)
                test_labels = test_labels.squeeze(0).to(device)
                #test_sub_id = test_sub_id.to(device)

                test_outputs = model(test_inputs)
                test_loss += pearson_loss(test_outputs, test_labels).mean()
                test_metric += pearson_metric(test_outputs, test_labels).mean()

            test_loss /= len(test_dataloader)
            test_metric /= len(test_dataloader)
            test_metric = test_metric.mean()    
            print(f'|-Test-|{epoch}: {test_loss.mean().item():.3f} {test_metric.item():.3f}')
            #writer.add_losses("Loss", "Test",  test_loss.mean().item(), epoch)
            #writer.add_losses("Pearson", "Test",  test_metric, epoch)

#     if epoch % 10 == 0:
#         learning_rate = print(optimizer.param_groups[0]["lr"])
#         save_checkpoint(model, optimizer, learning_rate, epoch, save_path)    

    scheduler.step()

YourModel(
  (fc): Linear(in_features=128, out_features=10, bias=True)
  (conv): Conv1d(64, 128, kernel_size=(7,), stride=(1,), padding=(3,))
)
Epoch 1
|-Train-|0: 0.341
|-Validation-|0: -0.006 0.006
|-Test-|0: -0.000 0.000
Epoch 2
|-Train-|1: 0.171
|-Validation-|1: -0.006 0.006
|-Test-|1: -0.004 0.004
Epoch 3
|-Train-|2: 0.216
|-Validation-|2: -0.006 0.006
|-Test-|2: -0.003 0.003
Epoch 4
|-Train-|3: 0.151
|-Validation-|3: -0.007 0.007
|-Test-|3: -0.001 0.001
Epoch 5
|-Train-|4: 0.201
|-Validation-|4: -0.009 0.009
|-Test-|4: -0.001 0.001
Epoch 6
|-Train-|5: 0.213
|-Validation-|5: -0.008 0.008
|-Test-|5: -0.001 0.001
Epoch 7
|-Train-|6: 0.196
|-Validation-|6: -0.007 0.007
|-Test-|6: -0.000 0.000
Epoch 8
|-Train-|7: 0.158
|-Validation-|7: -0.007 0.007
|-Test-|7: 0.001 -0.001
Epoch 9
|-Train-|8: 0.156
|-Validation-|8: -0.008 0.008
|-Test-|8: 0.001 -0.001
Epoch 10
|-Train-|9: 0.191
|-Validation-|9: -0.008 0.008
|-Test-|9: 0.001 -0.001
